In [128]:
import pandas as pd
from rapidfuzz.process import extractOne
import re
import os
import math


In [129]:

def load_projections(dir):
    positions = ["QB", "RB", "WR", "TE", "K", "DEF"]
    dfs = {}
    for pos in positions:
        file = pos + "_projections.csv"
        if os.path.exists(os.path.join(dir, file)):
            dfs[pos] = pd.read_csv(os.path.join(dir, file))
    return dfs

In [130]:
df_sleeper = pd.read_csv('data/sleeper/all_players.csv')
# print(df_sleeper.columns)
# print(df_sleeper["full_name"])

dict_sleeper = {
    "QB": df_sleeper.loc[df_sleeper['position'] == "QB"],
    "RB": df_sleeper.loc[df_sleeper['position'] == "RB"],
    "WR": df_sleeper.loc[df_sleeper['position'] == "WR"],
    "TE": df_sleeper.loc[df_sleeper['position'] == "TE"],
    "DEF": df_sleeper.loc[df_sleeper['position'] == "DEF"],
    "K": df_sleeper.loc[df_sleeper['position'] == "K"]
}
# dict_sleeper["QB"].columns
# pos="QB"
# dict_sleeper[pos].loc[:,["sleeper_id", "full_name", "team", "position"]]


In [131]:

# os.listdir("data/proj_scrape")

dict_dfs = {"QB": [], "RB": [], "WR": [], "TE": [], "DEF": [], "K": []}

dir = "data/proj_scrape/"

for root, dirs, files in os.walk(dir):
    for file in files:
        if re.match(r'[A-Z]{1,3}_projections.csv', file):
            df = pd.read_csv(os.path.join(root, file))
            data_src = re.sub(r'data/proj_scrape/', '', root)
            pos = re.sub(r'_projections.csv', '', file)
            dict_dfs[pos].append(df)
            

dict_dfs = {k: pd.concat(v).reset_index(drop=True) for k, v in dict_dfs.items()}

for pos, df in dict_dfs.items():
    # print(pos)
    df["name_match"] = df["player"].apply(lambda x: extractOne(x, dict_sleeper[pos]["full_name"], score_cutoff=80))
    df["src_player_name"] = df["player"]
    df["sleeper_row"] = df["name_match"].apply(lambda x: x[2] if x else None)
    # add sleeper_id to each df in dict_dfs
    df["sleeper_id"] = df["sleeper_row"].apply(lambda x: dict_sleeper[pos].loc[x, "player_id"] if x and not math.isnan(x) else None)
    # print(df.columns)
    df = df.drop(columns=["name_match", "sleeper_row", "player", "team", "pos"])
    tmp_slpr = dict_sleeper[pos].loc[:, ["sleeper_id", "full_name", "team", "position"]]
    tmp_slpr["sleeper_id"] = tmp_slpr["sleeper_id"].astype(str).apply(lambda x: re.sub(r'\.0', '', x))
    df["sleeper_id"] = df["sleeper_id"].astype(str).apply(lambda x: re.sub(r'\.0', '', x))
    df = df.merge(tmp_slpr, on="sleeper_id", how="inner")
    df.to_csv(f"data/projections/{pos}_projections.csv", index=False)
    

In [125]:
dict_dfs["WR"].shape

(871, 33)

In [126]:
df_mean_var = pd.read_csv('data/nflverse/mean_var_weekly_fp.csv')
dict_dfs_update = {}
for pos, df in dict_dfs.items():
    df = df.loc[:, ["sleeper_id", "player", "pos"]]
    df["name_match_idx"] = df["player"].apply(lambda x: extractOne(x, df_mean_var["player_display_name"], score_cutoff=80))
    df["name_match_idx"] = df["name_match_idx"].apply(lambda x: x[2] if x else None)
    dict_dfs_update[pos] = df.merge(df_mean_var, left_on="name_match_idx", right_index=True, how="inner")
    

In [127]:
dict_dfs_update["QB"].to_csv("data/nflverse/qb_mean_var.csv", index=False)
dict_dfs_update["RB"].to_csv("data/nflverse/rb_mean_var.csv", index=False)
dict_dfs_update["WR"].to_csv("data/nflverse/wr_mean_var.csv", index=False)
dict_dfs_update["TE"].to_csv("data/nflverse/te_mean_var.csv", index=False)
dict_dfs_update["DEF"].to_csv("data/nflverse/def_mean_var.csv", index=False)
dict_dfs_update["K"].to_csv("data/nflverse/k_mean_var.csv", index=False)

In [29]:
# import nfl_data_py as nfl

# df_weekly = nfl.import_weekly_data([2021, 2022, 2023])
# df_weekly['fantasy_points_halfppr'] = df_weekly['fantasy_points_ppr'] - .5*df_weekly['receptions']

Downcasting floats.


In [34]:
# df_weekly = df_weekly.loc[:, ["player_id", "player_display_name", "season", "week", "position", "fantasy_points_halfppr"]]
# df_weekly.columns

Index(['player_id', 'player_display_name', 'season', 'week', 'position',
       'fantasy_points_halfppr'],
      dtype='object')

In [28]:
# df_weekly.groupby(["player_display_name", "player_id", "season", "week"]).count()['player_name']

player_display_name  player_id   season  week
A.J. Brown           00-0035676  2021    1       1
                                         2       1
                                         3       1
                                         5       1
                                         6       1
                                                ..
Zonovan Knight       00-0037157  2022    16      1
                                         17      1
                                         18      1
                                 2023    3       1
                                         5       1
Name: player_name, Length: 16982, dtype: int64

In [45]:

# # os.listdir("data/proj_scrape")

# dict_dfs = {"QB": [], "RB": [], "WR": [], "TE": [], "DEF": [], "K": []}

# dir = "data/projections/"

# for root, dirs, files in os.walk(dir):
#     for file in files:
#         if re.match(r'[A-Z]{1,3}_projections.csv', file):
#             df = pd.read_csv(os.path.join(root, file))
#             data_src = re.sub(r'data/proj_scrape/', '', root)
#             pos = re.sub(r'_projections.csv', '', file)
#             dict_dfs[pos].append(df)
            

# dict_dfs = {k: pd.concat(v).reset_index(drop=True) for k, v in dict_dfs.items()}


# for pos, df in dict_dfs.items():
# #     # print(pos)
#     df_weekly_pos = df_weekly.loc[df_weekly["position"] == pos]
#     print(df_weekly_pos.shape)
#     df["name_match"] = df["full_name"].apply(lambda x: extractOne(x, df_weekly_pos['player_display_name'], score_cutoff=80))
#     # df["src_player_name"] = df["full_name"]
#     df["nflverse_row"] = df["name_match"].apply(lambda x: int(x[2]) if x else None)
#     df = df.merge(df_weekly_pos, left_on="nflverse_row", right_index=True, how="left")
# #     # add sleeper_id to each df in dict_dfs
#     # df["sleeper_id"] = df["sleeper_row"].apply(lambda x: dict_sleeper[pos].loc[x, "player_id"] if x and not math.isnan(x) else None)
# #     # print(df.columns)
# #     df = df.drop(columns=["name_match", "sleeper_row", "player", "team", "pos"])
# #     tmp_slpr = dict_sleeper[pos].loc[:, ["sleeper_id", "full_name", "team", "position"]]
# #     tmp_slpr["sleeper_id"] = tmp_slpr["sleeper_id"].astype(str).apply(lambda x: re.sub(r'\.0', '', x))
# #     df["sleeper_id"] = df["sleeper_id"].astype(str).apply(lambda x: re.sub(r'\.0', '', x))
# #     df = df.merge(tmp_slpr, on="sleeper_id", how="inner")
# #     df.to_csv(f"data/projections/{pos}_projections.csv", index=False)
    

(2077, 6)
(4269, 6)
(6878, 6)
(3337, 6)
(0, 6)
(1, 6)
